In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
# R.EnableImplicitMT()

ModuleNotFoundError: No module named 'cppyy'

In [2]:
# import os
# recompile = True
# try:
#     if os.path.getmtime('../Python/Particles_C.so') - os.path.getmtime('../Python/Particles.C') > 0:
#         recompile = False
#         print("Recompile is not needed")
#     else:
#         print("Recompiling: ")
# except:
#     print("Recompile needed, file not found.")
# if recompile:
#     R.gROOT.LoadMacro("../Python/Particles.C++")
# else:
#     R.gROOT.LoadMacro("../Python/Particles_C.so")
# R.Particles()

In [1]:
import sys
print(sys.path)

['/System/Volumes/Data/data/HPS/Analysis/PhysRun2021', '/System/Volumes/Data/data/HPS/Analysis', '/Users/maurik/miniforge3/envs/C/lib/python310.zip', '/Users/maurik/miniforge3/envs/C/lib/python3.10', '/Users/maurik/miniforge3/envs/C/lib/python3.10/lib-dynload', '', '/Users/maurik/miniforge3/envs/C/lib/python3.10/site-packages']


In [3]:
def fancy_plot(histo, ones_lb, opt=0):
    """
    Fancy plot of the Calorimeter with the 2D histo overlayed.
    """
    # this defines the position of the top right region of big boxes, others will fall in by symmetry
    ecal_x_first = 1
    ecal_nx_first = -1
    ecal_y_first = 1
    ecal_ny_first = -1

    ecal_nx = 23
    ecal_ny = 5

    xax = histo.GetXaxis()
    yax = histo.GetYaxis()

    # if R.gROOT.FindObject(histo.GetName()+"_oneslb"): # if this one exists all the rest probably exist too
    #     print(f'histogram {histo.GetName()+"_oneslb"} found.')
    #     ones_lb= R.gROOT.FindObject(histo.GetName()+"_oneslb")
    #     ones_lb.Clear()
    if ones_lb is None:
        print(f'Booking histogram {histo.GetName()+"_oneslb"}')
        ones_lb = R.TH2F(histo.GetName()+"_oneslb","oneslb",(ecal_nx+1)*2+1,-ecal_nx-1.5,ecal_nx+1.5,(ecal_ny+1)*2+1,-ecal_ny-1.5,ecal_ny+1.5);
    else:
        ones_lb.Clear()

    if opt& 0x4 ==0:
        histo.SetMaximum();
    if histo.GetMaximum() < 1:
        histo.SetMaximum(1.1)

    if opt & 0x6 == 0:
        histo.SetStats(0)

    SetMax=histo.GetMaximum()
    if SetMax<1.1:
        SetMax=1.1

    xax=ones_lb.GetXaxis();
    yax=ones_lb.GetYaxis();

    # this chunk of code just puts the grid in the right place
    for i in range(ecal_nx):
        for j in range(ecal_ny):
            ones_lb.SetBinContent(xax.FindBin(ecal_x_first+i),yax.FindBin(ecal_y_first+j),1)
            ones_lb.SetBinContent(xax.FindBin(ecal_x_first+i),yax.FindBin(ecal_ny_first-j),1)
            if (j==0 and 0 < i < 10):
                pass
            else:
                ones_lb.SetBinContent(xax.FindBin(ecal_nx_first-i),yax.FindBin(ecal_ny_first-j),1)
                ones_lb.SetBinContent(xax.FindBin(ecal_nx_first-i),yax.FindBin(ecal_y_first+j),1)

    ones_lb.Scale(SetMax) # scale them so the boxes are big enough
    # draw stuff
    xax.SetTitle("x crystal index")
    yax.SetTitle("y crystal index")
    if opt&0x1:
        histo.Draw("col")
    else:
        histo.Draw("colz")

    ones_lb.Draw("boxsame")
    return ones_lb

In [4]:
ch = R.TChain("MiniDST")
# ch.Add("/data/HPS/data/physrun2021/MC/hpsForward_gamma_3.0GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")

Number of events loaded:  36.588M


In [5]:
df = R.RDataFrame(ch)
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy     ecal_cluster_hits       ecal_cluster_nhits      ecal_cluster_seed_energy
ecal_cluster_seed_index ecal_cluster_seed_ix    ecal_cluster_seed_iy    ecal_cluster_time       
ecal_cluster_x          ecal_cluster_y          ecal_cluster_z          ecal_hit_energy         
ecal_hit_index_x        ecal_hit_index_y        ecal_hit_time           event_number            
ext_trigger             hodo_cluster_energy     hodo_cluster_ix         hodo_cluster_iy         
hodo_cluster_layer      hodo_cluster_time       hodo_hit_energy         hodo_hit_hole           
hodo_hit_index_x        hodo_hit_index_y        hodo_hit_layer          hodo_hit_time           
part_charge             part_ecal_cluster       part_energy             part_goodness_of_pid    
part_lcio_type          part_mass               part_pdg                part_px                 
part_py                 part_pz                 part_track              part_track_chi2        

In [6]:
ones = None
event=39
c_name = "cc"+str(event)
cc1 = R.TCanvas("cc1","Canvas",1000,600)

In [7]:
cc1.Clear()
dfr = df.Range(event,event+1)
p0 = dfr.Display(("run_number","event_number","ecal_cluster_nhits"))
p1 = dfr.Display(("ecal_cluster_energy","ecal_cluster_time","ecal_cluster_hits","ecal_cluster_seed_index"))
p2 = dfr.Display(("ecal_hit_energy","ecal_hit_time","ecal_hit_index_x","ecal_hit_index_y"))
h_ecal = dfr.Histo2D(("h_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5),"ecal_hit_index_x", "ecal_hit_index_y", "ecal_hit_energy")
# p0.Print()
# p1.Print()
# p2.Print()
h_draw = h_ecal.GetPtr()
ones = fancy_plot(h_ecal, ones, 0x0)
cc1.Draw()
event +=1

Booking histogram h_ecal_oneslb


In [8]:
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters

In [9]:
R.gSystem.Load("/data/HPS/lib/libMiniDst")

-2

Warning in <TFile::Init>: no StreamerInfo found in /System/Volumes/Data/data/HPS/lib/libMiniDst_rdict.pcm therefore preventing schema evolution when reading this file. The file was produced with version 6.29/01 of ROOT.
Error in <TMacOSXSystem::Load>: version mismatch, /data/HPS/lib/libMiniDst.dylib = 62901, ROOT = 63101


In [10]:
# Setup the MiniDST class, which makes for easier event by event data inplection.
# This MiniDST class looks directly into the TTree, so does not use the RDataframe class.
# You need to:
mdst = R.MiniDst()          # Initiate the class
mdst.use_mc_particles=True  # Tell it to look for the MC Particles in the TTree
mdst.DefineBranchMap()      # Define the map of all the branches to the contents of the TTree
mdst.SetBranchAddressesOnTree(ch) # Connect the TChain (which contains the TTree) to the class.

Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.


In [11]:
def make_ecal_snaphot(mini_dst, hist, opt=0):
    """Return a 2D histogram for the ECal, with the energy of the hits on the z-axis"""
    if hist is None:
        hist = R.TH2D("hist_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5)
    else:
        hist.Reset()
    # Fill the histogram by looping over the ECal hits
    for i in range(len(mini_dst.ecal_hit_index_x)):
        hist.Fill(mini_dst.ecal_hit_index_x[i], mini_dst.ecal_hit_index_y[i], mini_dst.ecal_hit_energy[i])
    return hist

In [12]:
cc2 = R.TCanvas("cc2","Canvas",1000,600)
hh = None
ones2 = None

In [20]:
ch.GetEntry(event)
hh = make_ecal_snaphot(mdst, hh, 0)
hh.SetStats(0)
cc2.Clear()
ones2 = fancy_plot(hh, ones2, 0x0)
# cc2.Update()
cc2.Draw()
event += 1

In [21]:
print(f"Run: {mdst.run_number}, Event:{mdst.event_number}")
print(f"NClusters: {len(mdst.ecal_cluster_seed_ix)} :",end="")
for i in range(len(mdst.ecal_cluster_seed_index)):
    print(f" [{mdst.ecal_cluster_seed_index[i]}]({mdst.ecal_cluster_x[i]:7.3f},{mdst.ecal_cluster_y[i]:7.3f},{mdst.ecal_cluster_energy[i]:7.3f})", end=",")
print("\n")
print("   i |   x |   y |  Energy  ")
print("----------------------------")
for i in range(len(mdst.ecal_hit_index_x)):
    print(f" {i:3d} | {mdst.ecal_hit_index_x[i]:3d} | {mdst.ecal_hit_index_y[i]:3d} | {mdst.ecal_hit_energy[i]:7.5f} ")

Run: 14700, Event:4793906
NClusters: 1 : [4](-105.012, 29.216,  1.843),

   i |   x |   y |  Energy  
----------------------------
   0 |  13 |   1 | 0.04145 
   1 |  -2 |   2 | 0.07294 
   2 | -10 |   2 | 0.02647 
   3 | -10 |   2 | 0.00427 
   4 | -11 |   1 | 1.21576 
   5 | -11 |   1 | 0.01550 
   6 | -11 |   2 | 0.09041 
   7 | -12 |   1 | 0.14659 
   8 | -12 |   2 | 0.04527 
   9 | -13 |   1 | 0.03085 
  10 | -18 |   1 | 0.03073 
  11 | -18 |   1 | 0.04071 
  12 | -23 |   5 | 0.01430 
  13 |   3 |  -1 | 0.02476 
  14 |   2 |  -1 | 0.02832 
  15 | -18 |  -1 | 0.03478 


In [22]:
print_mc_particle_tree(mdst)